# Feature extraction part 2

This notebook contains on from `12_feature_extraction.ipynb`, looking at replicating some of the other methods in the FHRMA toolbox.

In [1]:
# Import packages
from dataclasses import dataclass
import glob
from itertools import compress, groupby
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import pandas as pd
from scipy import io
from statistics import mode

In [2]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    fhrma_train_csv = './fhrma/train_test_data/traindata_csv/'
    fhrma_test_csv = './fhrma/train_test_data/testdata_csv/'


paths = Paths()

## Maeda et al. 2012 Baseline FHR

[Maeda et al. 2012](https://benthamopen.com/contents/pdf/TOMDJ/TOMDJ-4-28.pdf) - Central Computerized Automatic Fetal Heart Rate Diagnosis with a Rapid and Direct Alarm System

FHR was sampled every 250ms over a 5-minute period, and averaged every 2 seconds to determine 150 FHR (also found 150 uterine contraction data) (as there are 30 x 2 seconds in a minute, so 150 x 2 seconds in 5 minutes). FHR data were counted in intervals of 10 beats per minute (bpm) ranging from 0 to 200 bpm. The data in the interval with the most frequent FHR data was then averaged to determine the FHR baseline. 

So basically...

1. **Find the average of every 2 seconds**

2. **Look at data from a five minute period** - this will mean you are looking at a sample of 150 FHR (as each represents average of 2 seconds, and there are 150 x 2 seconds in 5 minutes)

3. **Look at frequency of data in bins of 10bpm** - i.e. number of FHR that are 140-149.99, 150-150.99, and so on.

4. **Find the most frequent bin** - for example, 140-150 has the most records, then just use the data from that bin

5. **Find the average of the heartrates from that bin** - so might get a result like 145.5, or so on. That represents the baseline FHR for that 5 minute portion of the data.

### MATLAB Implementation

Boudet et al. implement this method [in the FHRMA toolbox using MATLAB](https://github.com/utsb-fmm/FHRMA/blob/master/aammaeda.m), and this is copied below:

```
sFHR=avgsubsamp(FHR,8);
baseline=zeros(1,length(FHR));

for win=[0:150:length(sFHR)-151 length(sFHR)-150]
    
    bins=zeros(1,25);

    for i=1:150
        bins(ceil(sFHR(win+i)/10))=bins(ceil(sFHR(win+i)/10))+1;
    end
    [~,bestbins]=max(bins(1:20));
    
    baseline(win*8+1:win*8+1200)=mean(sFHR( sFHR<=bestbins*10 & sFHR>(bestbins-1)*10 ));

end


baseline(win*8+1201:length(FHR))=baseline(win*8+1200);
```

They use a function `avgsubsamp` for subsampling by average, which is also copied below:

```
function y=avgsubsamp(x,factor)
    y=zeros(1,floor(length(x)/factor));
    for i=1:length(y)
        y(i)=mean(x((i-1)*factor+1:i*factor));
    end
end
```

### Python Implementation

#### Set up

Load the FHRMA data - the raw FHR trace, and the results from their implementation of this method on that same trace.

In [3]:
# Load the FHR for train01
fhr = pd.read_csv(os.path.join(paths.fhrma_train_csv, 'train01.csv'),
                  header=None)[0].values
fhr[0:10]

array([168., 168., 168., 170., 170., 170., 172., 172., 172., 173.])

In [4]:
# Load FHRMA version of results
md_std = io.loadmat('./fhrma/MD_std.mat')

# Get array listing filenames (and hence order of the data)
fhrma_files = np.concatenate(np.concatenate(md_std['data']['filename']))

# Get array with the baseline signal as per Maeda when implemented in FHRMA
fhrma_md = np.concatenate(md_std['data']['baseline'])

# Convert array into dictionary so each record is accompanied by relevant name
fhrma_maeda = {
    fhrma_files[i].replace('.fhr', ''): 
    fhrma_md[i][0] for i in range(len(fhrma_files))}

# Extract the same result as I am currently processing
fhrma_result = fhrma_maeda['train01']

#### Define function that replicates Maeda et al. 2012

Steps:

1. **Convert to average of every 2 seconds** - In FHRMA, they seperate the FHR into chunks of 8 (i.e. first 8 records, then next 8, then next 8, and so on). They then find the mean of each of those chunks.

2. **Find the most common heartrate bin in each 5 minute interval** - We're looking at every 5 minutes / 300 seconds (which equates to 150 of the 2 second results). We sort the heartrates into bins of 10bpm (e.g. 130-140, 140-150, 150-160), then look to see which bin is most common for that 5 minute period. FHRMA then find the mean of all heartrates from that bin across the entire recorded FHR CTG, but I am minded to suggested that this should be the mean of only the heartrates from that bin in the current five minute interval.

Note: This doesn't clean FHR beforehand, so could include large periods of 0, and includes values outside of normal.

In [5]:
def get_baseline(fhr, match_fhrma, show_process=False):
    '''
    Get FHR baseline using method from Maeda et al. 2012.

    Parameters
    ----------
    fhr : array
        Raw FHR sampled at 4Hz
    match_fhrma : boolean
        Whether to use a method that ensures results match FHRMA implementation
        of Maeda's method, or whether to use the method that I currently think
        best matches the description/intention in the original paper
    show_process : boolean
        Whether to print results as move through this process

    Return
    ------
    baseline : array
        Array where raw FHR is replaced by the baseline FHR calculated for the
        five minute interval which it belonged to
    '''
    # Create array of zeros of length of fhr, which will amend to store baseline
    baseline = [0] * len(fhr)

    # Find mean of every 8 records, generating shorter version of FHR (sfhr)
    sfhr = []
    start=0
    end=len(fhr)
    step=8
    for i in range(start, end, step):
        sfhr.append(np.mean(fhr[i:i+step]))

    # Print example of results
    if show_process:
        print(f'First 16 records in raw FHR: {fhr[:16]}')
        print(f'First 2 records in shortened FHR (average of each 8): {sfhr[:2]}')

    # Split the record into 5 minute intervals (if last less than 5 min, will be 
    # smaller). Loop through each interval
    for i in range(0, len(sfhr), 150):
        # Filter data to that 5-minute segment
        current = sfhr[0+i:150+i]
        # Convert each record into their upper bin boundary (e.g. 149 --> 150)
        bins = [math.ceil(x/10)*10 for x in current]
        # Find the most common bin
        most_common = mode(bins)

        # If want to match results in FHRMA, filter to records from that bin
        # from across the entire CTG
        if match_fhrma:
            mask = [(x <= most_common) & (x > most_common-10) for x in sfhr]
            filtered = list(compress(sfhr, mask))
        # Otherwise, filter to records in that bin from only that 5min interval
        else:
            mask = [(x <= most_common) & (x > most_common-10) for x in current]
            filtered = list(compress(current, mask))

        # Find mean of those filtered records in that bin
        mean = np.mean(filtered)

        # Set the records in baseline for that five minute interval to this mean
        baseline[0+(i*8):1200+(i*8)] = [mean] * 1200

    # Trim results to match original FHR length (as will overfill final small interval)
    baseline = baseline[:len(fhr)]

    # Print example of results from final run of that loop above
    if show_process:
        print(f'First 10 records of one of the intervals: {current[:10]}')
        print(f'Conversion of those records to bins: {bins[:10]}')
        print(f'Most common bin in the whole 5 minute interval: {most_common}')
        print(f'Calculated baseline FHR for that interval: {mean}')

    return(baseline)

#### Compare to FHRMA

Example where print results from throughout process, to help see/understand what calculations were performed...

In [6]:
python_result_nomatch = get_baseline(fhr, match_fhrma=False, show_process=True)

First 16 records in raw FHR: [168. 168. 168. 170. 170. 170. 172. 172. 172. 173. 173. 173. 172. 172.
 172. 171.]
First 2 records in shortened FHR (average of each 8): [169.75, 172.25]
First 10 records of one of the intervals: [150.25, 149.875, 146.5, 148.0, 147.75, 146.0, 149.75, 154.375, 157.875, 160.0]
Conversion of those records to bins: [160, 150, 150, 150, 150, 150, 150, 160, 160, 160]
Most common bin in the whole 5 minute interval: 160
Calculated baseline FHR for that interval: 155.7343205574913


Comparison of results to FHRMA. If using the method where I intend to match FHRMA, I get the same results for all expect the 150-160 bin, for which they have a different calculated mean. I have double-checked in my data below, and that is definitely the mean I get from values in that bin, so I am not certain why this difference arises.

If I don't intend to match FHRMA, but instead use the method which I currently think matches the paper (mean is from 5 min interval rather than whole record), then (as expected) I don't get matching results to FHRMA.

In [7]:
# Result from above where match_fhrma=False
[(k, sum(1 for i in g)) for k,g in groupby(python_result_nomatch)]

[(173.260593220339, 1200),
 (164.96959459459458, 1200),
 (165.3443396226415, 1200),
 (165.97619047619048, 1200),
 (175.21381578947367, 1200),
 (173.18489583333334, 1200),
 (165.03532608695653, 1200),
 (165.53605769230768, 1200),
 (155.54444444444445, 1200),
 (166.24576271186442, 1200),
 (154.58653846153845, 1200),
 (155.7343205574913, 807)]

In [8]:
# Results when match_fhrma=True
python_result_match = get_baseline(fhr, match_fhrma=True, show_process=False)
[(k, sum(1 for i in g)) for k,g in groupby(python_result_match)]

[(173.36320754716982, 1200),
 (165.35695043103448, 3600),
 (173.36320754716982, 2400),
 (165.35695043103448, 2400),
 (155.51384274640088, 1200),
 (165.35695043103448, 1200),
 (155.51384274640088, 2007)]

In [9]:
# View counts of consecutive equal values from FHRMA
[(k, sum(1 for i in g)) for k,g in groupby(fhrma_result)]

[(173.36320754716982, 1200),
 (165.35695043103448, 3600),
 (173.36320754716982, 1200),
 (165.35695043103448, 3600),
 (155.50259067357513, 1200),
 (165.35695043103448, 1200),
 (155.50259067357513, 2007)]

In [10]:
# Create shortenered FHR as in function above
sfhr = []
start=0
end=len(fhr)
step=8
for i in range(start, end, step):
    sfhr.append(np.mean(fhr[i:i+step]))

# Convert to array
sfhr = np.array(sfhr)

# Check the mean of all values in that bin
sfhr[(sfhr > 150) & (sfhr <= 160)].mean()

155.51384274640088